# LM KBC

## Load Dataset

In [ ]:
from kgat.data import load_id2map

In [ ]:
from kgat.data import load_json, LMKBCDataset
lmkbc_data_path = "./data/lm-kbc/conceptnet/proc/val.json"
lmkbc_entity_path = "./data/lm-kbc/conceptnet/proc/entities.txt"
lmkbc_relations_path = "./data/lm-kbc/conceptnet/proc/relations.txt"
lmkbc_triples_path = "./data/lm-kbc/conceptnet/proc/triples.json"

lmkbc_id2entity = load_id2map(lmkbc_entity_path)
lmkbc_id2rel = load_id2map(lmkbc_relations_path)
lmkbc_triples = load_json(lmkbc_triples_path)

In [ ]:
from kgat.utils import Mask

prompt_template = f"Based on {Mask.KG_MASK} complete this -> S : {Mask.SUBJECT_MASK} , R : {Mask.RELATION_MASK} , O : {Mask.OBJECT_MASK}"
graph_query_template = f"S : {Mask.SUBJECT_MASK} , R : {Mask.RELATION_MASK}"

In [ ]:
lmkbc_ds = LMKBCDataset(lmkbc_data_path, lmkbc_id2entity, lmkbc_id2rel, lmkbc_triples, prompt_template=prompt_template, graph_query_template=graph_query_template)

In [ ]:
tokenizer.add_tokens(Mask.KG_MASK)

1

In [ ]:
kg_id = tokenizer.get_added_vocab()[Mask.KG_MASK]

## Prepare Model

In [ ]:
from kgat.model.text import TextModule
from kgat.model.text import SubgraphVTTransformation
from kgat.model import ModelForLMKBC

vt_transformer = SubgraphVTTransformation(subgraph_emb_dim=1024, hidden_dim=1024, word_emb_dim=768)
text_module = TextModule(vt_transformer=vt_transformer,
                         clm_embedding=clm.transformer.wte,
                         clm_model=clm,
                         kg_id=kg_id)
lmkbc_model = ModelForLMKBC(graph_module=graph_module, text_module=text_module)

## Forward Pass

In [ ]:
from kgat.data import LMKBCCollator

lmkbc_collator = LMKBCCollator(tokenizer=tokenizer)
lmkbc_dataloader = DataLoader(lmkbc_ds, batch_size=2, shuffle=False, collate_fn=lmkbc_collator)

In [ ]:
import time

start = time.time()
batch = next(iter(lmkbc_dataloader))

print(time.time() - start)

Token indices sequence length is longer than the specified maximum sequence length for this model (1298 > 1024). Running this sequence through the model will result in indexing errors


4.084503412246704


In [ ]:
batch["labels"].shape

torch.Size([2, 1298])

In [ ]:
# """graph_query_input_ids, graph_query_attention_mask,
# prompt_input_ids, prompt_attention_mask,
# entities_input_ids, entities_attention_mask,
# relations_input_ids, relations_attention_mask,
# x_coo, batch"""

In [ ]:
# text_in, graph_query, entities, relations, x_coo, batch, text_out

In [ ]:
# prompt = tokenizer(batch[0], padding=True, truncation=True, max_length=64, return_tensors="pt")
# graph_query = tokenizer(batch[1], padding=True, truncation=True, max_length=64, return_tensors="pt")
# entities = tokenizer(batch[2], padding=True, truncation=True, max_length=16, return_tensors="pt")
# relations = tokenizer(batch[3], padding=True, truncation=True, max_length=16, return_tensors="pt")
# x_coo = batch[4]
# node_batch = batch[5]
# labels = tokenizer(batch[6], padding=True, truncation=True, max_length=64, return_tensors="pt")

In [ ]:
labels = batch.pop("labels")

In [ ]:
out, mean_fused_score, edge_batch = lmkbc_model(**batch)

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3422945280 bytes.